In [1]:
## 패키지/라이브러리 설치
# %pip install python-docx tiktoken chromadb

In [6]:
import os
import tiktoken
import chromadb
from docx import Document
from openai import OpenAI
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction


# 1. 문서내용 읽기(python-docx) ==============================================
document = Document('law_1.docx')

document.paragraphs[1].text
for index, paragraphs in enumerate(document.paragraphs):
    print(f'{index}. {paragraphs.text}')

    if index == 4:
        break

## 1.1 읽어 온 단락을 하나의 문서로 만들기
full_text = ''

for paragraphs in document.paragraphs:
    full_text += f'{paragraphs.text}'

# 2. 문서 분할(쪼개기) =======================================================
encoding_model = tiktoken.encoding_for_model('gpt-4o')

## 2.1 토큰화
encoding = encoding_model.encode(full_text)

def split_text(full_text, chunk_size):
    encoder_model = tiktoken.encoding_for_model('gpt-4o')
    total_encoding = encoder_model.encode(full_text)
    total_token_count = len(total_encoding)

    text_list = []
    for i in range(0, total_token_count, chunk_size):
        chunk = total_encoding[i: i+chunk_size]
        decoding = encoder_model.decode(chunk)
        text_list.append(decoding)
    return text_list
    
chunk_list = split_text(full_text, 1500)

# 3. 벡터 데이터베이스에 저장 =================================================
## 3.1 임베딩 모델 지정

OPEN_API_KEY = os.getenv('OPENAI_API_KEY')
openai_embedding = OpenAIEmbeddingFunction(
    model_name="text-embedding-3-large",
    api_key= OPEN_API_KEY,
)

## 3.2 크로마 클라이언트 생성
chroma_client = chromadb.Client()
collection_name = 'law_collection'

law_collection = chroma_client.get_or_create_collection(
    collection_name, 
    embedding_function=openai_embedding,
    )

## 3.3 컬렉션에 설정할 인덱스 생성
id_list= []

for index in range(len(chunk_list)):
    id_list.append(f'{index}')

law_collection.add(documents=chunk_list, ids=id_list)


## 3.4 chromadb에 생성된 컬렉션 목록 확인
collection = chroma_client.list_collections

# 4. 질문이 있으면, 벡터 데이터베이스에서 유사도 검색 ==========================
query = '전세사기피해자 금융지원에 대해 설명해주세요.'

## 4.1 유사도 검색 [방법 1]
retrieved_doc = law_collection.query(query_texts=query, n_results=3)
client = OpenAI()

response1 = client.responses.create(
    model = 'gpt-4o',
    instructions = f'''
        당신은 전세사기 피해 법률 전문가입니다.
        [context]를 참고하여 사용자의 질문에 답변해주세요.
        [context]

        {retrieved_doc['documents'][0]}
        ''',
    input= query
)

print(response1.output_text)

## 4.2 유사도 검색 [방법 2]
# response2 = client.responses.create(
#     model="gpt-4o",
#     input=[
#         {
#             "role": "developer",
#             "content": f'''당신은 전세사기 피해 법률 전문가입니다.
#         {retrieved_doc['documents'][0]}를 참고하여 사용자의 질문에 답변해주세요.
#         '''
#         },
#         {
#             "role": "user",
#             "content": query
#         },
#     ]
# )

# print(response2.output_text)

0. 
1. 전세사기피해자 지원 및 주거안정에 관한 특별법 ( 약칭: 전세사기피해자법 )
2. [시행 2024. 11. 11.] [법률 제20429호, 2024. 9. 10., 일부개정]
3. 
4. 국토교통부(피해지원총괄과) 044-201-5233, 5234
전세사기피해자에 대한 금융지원은 전세사기로 인해 피해를 입은 임차인의 주거 안정을 돕기 위해 마련된 지원책입니다. 이러한 지원은 주로 임차보증금 반환을 못 받아 어려움을 겪는 피해자를 대상으로 하며, 주로 다음과 같은 형태로 제공됩니다:

1. **저금리 대출 지원**: 피해자에게 저금리로 긴급 주거안정 자금을 대출해 주어 경제적 부담을 완화합니다.

2. **보증금 반환 보장**: 임차보증금 반환이 어려울 경우, 국가에서 해당 보증금을 일정 부분 보장해 줍니다.

3. **주택 구입 및 임대 지원**: 새로운 주택 구입이나 임대 시 우선권을 부여하거나 금융적 지원을 제공합니다.

이러한 금융지원은 전세사기피해자 지원 및 주거안정에 관한 특별법에 의해 규정되며, 구체적인 지원 내용과 조건은 국토교통부장관이 정한 지침에 따르게 됩니다. 피해자는 관련 위원회의 심의·의결을 거쳐 국토교통부장관이 결정하게 됩니다.
